In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, not_
from dotenv import load_dotenv
import os
import sys
import requests
from typing import Any
from bs4 import BeautifulSoup
import time
import re
import random
import json
from datetime import datetime

from models import Player, GameStats, KtcValue

In [3]:
load_dotenv()
DB_USERNAME = os.environ['DB_USERNAME']
DB_PASSWORD = os.environ['DB_PASSWORD']
DB_HOST = os.environ['DB_HOST']
DB_PORT = os.environ['DB_PORT']
DB_NAME = os.environ['DB_NAME']

DB_URL = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DB_URL, echo=False)

Session = sessionmaker(bind=engine)
session = Session()

In [5]:
player_to_ktc = pd.read_excel("keeptradecut (2).xlsx")

In [6]:
for i, row in player_to_ktc.iterrows():
    if not pd.isna(row['url']):
        player_id = row['url'].rsplit('/', 1)[-1]
        player_obj = session.query(Player).filter_by(sleeper_id=str(row['sleeper_id'])).first()
        
        player_obj.ktc_id = player_id
        session.commit()

In [7]:
def fetch_soup(url) -> BeautifulSoup:
    time.sleep(random.uniform(1,5))
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    return soup

def find_value_data(soup) -> list[dict]:
    pattern = re.compile(r'var playerSuperflex', re.MULTILINE | re.DOTALL)
    
    content = soup.find('script', text=pattern)
    content = str(content)
    bottomHalf = content.split("var playerSuperflex = ",1)[1]
    fullCut = bottomHalf.split('var playerOneQB = ',1)[0].strip()
    json_text= fullCut[:-1]
    js = json.loads(json_text)
    data = js['overallValue']
    return data

def scrape(sleeper_id: int, url):
    player = session.query(Player).filter_by(sleeper_id=str(sleeper_id)).first()

    if player:
        player_soup = fetch_soup(url)
        ktc_value_history = find_value_data(player_soup)
        
        for ktc_value in ktc_value_history:
            date = datetime.strptime(ktc_value['d'], '%Y-%m-%d')
            value = int(ktc_value['v'])
            ktc_value_obj = KtcValue(
                player_id = player.id,
                date = date,
                value = value
            )
            session.merge(ktc_value_obj)
        session.commit()
        print(f'{player.display_name} KTC data commited')

In [8]:
player_to_ktc.dropna(inplace=True)

In [9]:
for i, row in player_to_ktc.iterrows():
    scrape(row['sleeper_id'], row['url'])

C:\Users\simer\AppData\Local\Temp\ipykernel_10760\2970735583.py:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  content = soup.find('script', text=pattern)


Tyjae Spears KTC data commited
Ryan Fitzpatrick KTC data commited
Drake Maye KTC data commited
MarShawn Lloyd KTC data commited
Zach Charbonnet KTC data commited
Jayden Daniels KTC data commited
Dalton Kincaid KTC data commited
Jaylen Wright KTC data commited
Xavier Legette KTC data commited
Jamari Thrash KTC data commited
Will Levis KTC data commited
Anthony Richardson KTC data commited
Jonathon Brooks KTC data commited
Jaheim Bell KTC data commited
Brock Bowers KTC data commited
J.D. McKissic KTC data commited
Brian Thomas KTC data commited
Kellen Mond KTC data commited
Roschon Johnson KTC data commited
Marvin Mims KTC data commited
Jared Cook KTC data commited
Michael Mayer KTC data commited
John Brown KTC data commited
Shi Smith KTC data commited
James Cook KTC data commited
Trey McBride KTC data commited
Jahmyr Gibbs KTC data commited
Josh Downs KTC data commited
Tank Bigsby KTC data commited
Mike Boone KTC data commited
Cole Beasley KTC data commited
Marvin Harrison KTC data comm

In [ ]:
player = player = session.query(Player).filter_by(sleeper_id="1825").first()
ktc_count = len(player.ktc_values) if player.ktc_values else 0
print(f"{player.display_name}: {ktc_count}")

Jarvis Landry: 628
